## Deliverable 2. Create a Customer Travel Destinations Map.

In [8]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,2022-02-04 01:58:35,-9.8000,-139.0333,78.91,72,4,16.35,clear sky
1,1,Kapaa,US,2022-02-04 01:57:23,22.0752,-159.3190,78.78,66,0,1.99,clear sky
2,2,Soyo,AO,2022-02-04 01:56:43,-6.1349,12.3689,77.20,75,93,3.04,overcast clouds
3,3,Sylvania,US,2022-02-04 01:59:20,41.7189,-83.7130,21.04,89,100,7.00,overcast clouds
4,4,General Pico,AR,2022-02-04 01:59:20,-35.6566,-63.7568,73.81,61,53,7.99,light rain


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atuona,PF,2022-02-04 01:58:35,-9.8000,-139.0333,78.91,72,4,16.35,clear sky
1,1,Kapaa,US,2022-02-04 01:57:23,22.0752,-159.3190,78.78,66,0,1.99,clear sky
2,2,Soyo,AO,2022-02-04 01:56:43,-6.1349,12.3689,77.20,75,93,3.04,overcast clouds
4,4,General Pico,AR,2022-02-04 01:59:20,-35.6566,-63.7568,73.81,61,53,7.99,light rain
6,6,Bundaberg,AU,2022-02-04 01:59:21,-24.8500,152.3500,84.22,63,75,3.00,broken clouds
...,...,...,...,...,...,...,...,...,...,...,...
708,708,Montepuez,MZ,2022-02-04 01:55:29,-13.1256,38.9997,69.12,95,100,3.38,overcast clouds
709,709,La Plata,AR,2022-02-04 02:00:01,-34.9215,-57.9545,73.99,87,57,7.09,broken clouds
713,713,Irece,BR,2022-02-04 02:03:52,-11.3042,-41.8558,71.08,66,25,8.23,scattered clouds
716,716,Narrabri,AU,2022-02-04 02:03:53,-30.3167,149.7833,78.53,44,20,17.27,few clouds


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284 entries, 0 to 717
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              284 non-null    int64  
 1   City                 284 non-null    object 
 2   Country              284 non-null    object 
 3   Date                 284 non-null    object 
 4   Lat                  284 non-null    float64
 5   Lng                  284 non-null    float64
 6   Max Temp             284 non-null    float64
 7   Humidity             284 non-null    int64  
 8   Cloudiness           284 non-null    int64  
 9   Wind Speed           284 non-null    float64
 10  Current Description  284 non-null    object 
dtypes: float64(4), int64(3), object(4)
memory usage: 26.6+ KB


In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,78.91,clear sky,-9.8000,-139.0333,
1,Kapaa,US,78.78,clear sky,22.0752,-159.3190,
2,Soyo,AO,77.20,overcast clouds,-6.1349,12.3689,
4,General Pico,AR,73.81,light rain,-35.6566,-63.7568,
6,Bundaberg,AU,84.22,broken clouds,-24.8500,152.3500,
7,Labuhan,ID,81.81,broken clouds,-6.8844,112.2051,
9,Baripada,IN,69.15,clear sky,21.9350,86.7214,
12,Inhambane,MZ,76.17,clear sky,-23.8650,35.3833,
15,Kavieng,PG,82.87,overcast clouds,-2.5744,150.7967,
16,Fuerte Olimpo,PY,78.03,overcast clouds,-21.0415,-57.8738,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] !=""]


In [32]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atuona,PF,78.91,clear sky,-9.8000,-139.0333,Villa Enata
1,Kapaa,US,78.78,clear sky,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
2,Soyo,AO,77.20,overcast clouds,-6.1349,12.3689,Hotel Mpampa
4,General Pico,AR,73.81,light rain,-35.6566,-63.7568,Euskadi Hotel
6,Bundaberg,AU,84.22,broken clouds,-24.8500,152.3500,Takalvan Motel
...,...,...,...,...,...,...,...
708,Montepuez,MZ,69.12,overcast clouds,-13.1256,38.9997,PembaAparthotel
709,La Plata,AR,73.99,broken clouds,-34.9215,-57.9545,Hotel Corregidor
713,Irece,BR,71.08,scattered clouds,-11.3042,-41.8558,Caraíbas Palace Hotel
716,Narrabri,AU,78.53,few clouds,-30.3167,149.7833,Crossroads Hotel


In [33]:
# 8a. Create the output File (CSV)
# Create the output file (CSV).
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [36]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [39]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))